# Day 21 Monkey Math

Or should it be called "monkey dispatch"? Cos that's my approach: register monkeys for events, and dispatch when event occurs

In [38]:
testData = """root: pppw + sjmn
dbpl: 5
cczh: sllz + lgvd
zczc: 2
ptdq: humn - dvpt
dvpt: 3
lfqf: 4
humn: 5
ljgn: 2
sjmn: drzm * dbpl
sllz: 4
pppw: cczh / lfqf
lgvd: ljgn * ptdq
drzm: hmdt - zczc
hmdt: 32"""

class MonkeyTalk:
    def __init__(self, input:str):
        self.input = input
    
    def restart(self):
        self.monkeys = []
        self.knownValues:{str:int} = {}
        self.monkeyListeners:{str:[Monkey]} = {}
        self.monkeyEvents=[]
        self.endTalk = False
        for l in self.input.splitlines():
            chunk = l.split(': ')
            monkeyID = chunk[0]
            operators = chunk[1].split(' ')
            if len(operators)==1:
                #number already known
                #m = Monkey(monkeyID, self)
                #The monkey will only shout and then do nothing, so don't think I even need to create the monkey object
                self.knownValues[monkeyID] = int(operators[0])
                self.chat(monkeyID)
            else:
                #number not already known
                m = Monkey(monkeyID,self)
                self.monkeys.append(m)
                m.left = operators[0]
                m.operator = operators[1]
                m.right = operators[2]
                self.registerListener(m.left, m)
                self.registerListener(m.right, m)
        #print('MonkeyTalk initalised with '+str(len(self.monkeys))+' monkeys listening and ' + str(len(self.monkeyEvents))+ ' monkeys already shouting')



    def registerListener(self, listenID:str, dispatchMonkey):
        if listenID in self.monkeyListeners:
            self.monkeyListeners[listenID].append(dispatchMonkey)
        else:
            self.monkeyListeners[listenID] = [dispatchMonkey]

    def chat(self, monkeyID:str):
        self.monkeyEvents.append(monkeyID)

    def runTalk(self,valueForHuman:int):
        self.restart()
        #run the event queue
        self.knownValues['humn'] = valueForHuman
        while self.monkeyEvents and not self.endTalk:
            e = self.monkeyEvents.pop(0)
            for m in self.monkeyListeners[e]:
                m.handleEvent() #we don't call any parameters, because the monkey will need to check both values exist anyway
        else:
            #print('Monkeys have run out of chat')
            pass
        return self.delta

class Monkey:
    def __init__(self, id:str, talk:MonkeyTalk):
        self.id = id
        self.talk = talk
        #setup in monkeytalk:
        #m.left = operators[1]
        #m.operator = operators[2]
        #m.right = operators[3]

    def handleEvent(self):
        #we know value has been updated, but need to check which
        if self.left in self.talk.knownValues and self.right in self.talk.knownValues:
            left = self.talk.knownValues[self.left]
            right = self.talk.knownValues[self.right]
            if self.id == 'root':
                #print('Root value found. Left='+str(left)+' Right='+str(right))
                #print('Root ='+str(left==right))
                self.talk.endTalk = True
                self.talk.delta = left - right
                print('delta: '+str(left-right))
            else:
                match self.operator:
                    case '+':
                        value = left + right
                    case '-':
                        value = left - right
                    case '*':
                        value = left * right
                    case '/':
                        value = left / right
                    case _:
                        raise Exception('unknown operator: '+str(self.operator))
                #print('Monkey ' + self.id + ' knowns left='+str(self.left)+' and right='+str(self.right)+' using operator='+self.operator+' yields value='+str(value))
                self.talk.knownValues[self.id] = value
                self.talk.chat(self.id)
            

#tests
tmt = MonkeyTalk(testData)
tmt.runTalk(1)


        


delta: -150.0


-150.0

In [30]:
puzzleinput = open('day21input.txt').read()
mt = MonkeyTalk(puzzleinput)
mt.runTalk(1000000000)


MonkeyTalk initalised with 1377 monkeys listening and 1378 monkeys already shouting
Root value found. Left=113704496362787.78 Right=56517685690674.0
Root =False
Monkeys have run out of chat


57186810672113.78

In [32]:
#let's throw some heavy weaponary at this
import sys
!{sys.executable} -m pip install scipy



Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 2.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 2.8 MB/s eta 0:00:0000:0100:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [46]:
from scipy.optimize import root_scalar
mt2 = MonkeyTalk(puzzleinput)
sol = root_scalar(mt2.runTalk, x0=170237589447588, x1=862)
sol.root, sol.iterations, sol.function_calls

delta: -2565717678594427.0
delta: 57202218066240.0
delta: 0.0


(3712643961892.0, 2, 3)